In [ ]:
#!Python3
# effective_panas.py - a program to practice the usage of pandas library

In [1]:
import numpy as np
import pandas as pd

In [31]:
#Chapter 7 - aggregation methods (data source in chapter 5)
url = 'https://github.com/mattharrison/datasets/raw/master/data/'\
      'vehicles.csv.zip'
df = pd.read_csv(url)
city_mpg = df.city08 # get city mpg list from df
highway_mpg = df.highway08 # get highway mpg list from df

/var/folders/_x/bt6gw2851b50zn7352f1qs_40000gn/T/ipykernel_68703/1888316542.py:4: DtypeWarning: Columns (68,70,71,72,73,74,76,79) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


In [32]:
#1. Find the count of non-missing values of a series
city_mpg.agg('count')

41144

In [33]:
#2. Find the number of entries in the data set
city_mpg.agg('size')

41144

In [34]:
#3. Find the number of unique entries in the list
city_mpg.agg('nunique')

105

In [35]:
#4. Find the mean of the series
city_mpg.mean()

18.369045304297103

In [36]:
#4b. Use agg to find the mean of the series
city_mpg.agg('mean')

18.369045304297103

In [37]:
#5. Find the max of the series
city_mpg.agg('max')

150

In [43]:
#Chapter 8
city_mpg = city_mpg.astype('int64').astype('category')
city_mpg

0        19
1         9
2        23
3        10
4        17
         ..
41139    19
41140    20
41141    18
41142    18
41143    16
Name: city08, Length: 41144, dtype: category
Categories (105, int64): [6, 7, 8, 9, ..., 137, 138, 140, 150]

In [47]:
sorted(set(city_mpg))

[6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 61,
 62,
 73,
 74,
 77,
 78,
 79,
 80,
 81,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 97,
 98,
 101,
 102,
 104,
 106,
 107,
 110,
 112,
 114,
 115,
 118,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 131,
 132,
 136,
 137,
 138,
 140,
 150]

In [45]:
values = pd.Series(sorted(set(city_mpg)))
city_type = pd.CategoricalDtype(categories=values,
            ordered=True)
city_mpg.astype(city_type)

0        19
1         9
2        23
3        10
4        17
         ..
41139    19
41140    20
41141    18
41142    18
41143    16
Name: city08, Length: 41144, dtype: category
Categories (105, int64): [6 < 7 < 8 < 9 ... 137 < 138 < 140 < 150]

In [58]:
#1. Convert numeric column to smaller type
highway_mpg_int16 = highway_mpg.astype('int16').memory_usage(deep=True)

In [59]:
#2. Calculate memory savings by doing so
highway_mpg.memory_usage(deep=True) - highway_mpg_int16

246864

In [71]:
#3 Convert string column into categorical type
highway_mpg_str = highway_mpg.astype('str')
cats = pd.CategoricalDtype(set(highway_mpg_str))
highway_mpg_cat = highway_mpg_str.astype(cats)
highway_mpg_cat

0        25
1        14
2        33
3        12
4        23
         ..
41139    26
41140    28
41141    24
41142    24
41143    21
Name: highway08, Length: 41144, dtype: category
Categories (92, object): ['34', '93', '13', '40', ..., '64', '24', '41', '82']

In [75]:
#4. Calculate memory savings
mb = highway_mpg_str.memory_usage(deep=True) - highway_mpg_cat.memory_usage(deep=True)
percent = mb / highway_mpg_str.memory_usage(deep=True)
print(mb, ' ', percent)

2378874   0.9798899779420397


In [ ]:
# Chapter 9

In [88]:
%%timeit
#1. Create a series from a numeric column that is 'high'
# if above the mean or 'low' if below using .apply
highway_mean = highway_mpg.mean()
def highlow(val):
    if val < highway_mean:
        return 'low'
    else:
        return 'high'
highway_mpg.apply(highlow)

7.83 ms ± 633 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [89]:
%%timeit
#2. Create a series from a numeric column that has the
# value 'high' if above or 'low' if below the mean
# using np.select
high = highway_mpg.gt(highway_mpg.mean())
pd.Series(np.select([high], ['high'], 'low'), index=highway_mpg.index)

4.49 ms ± 954 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [99]:
#3. Replace the missing values of a numeric series with the median
cyl = df.cylinders
cyl[cyl.isna()].fillna(cyl.median())

7138     6.0
7139     6.0
8143     6.0
8144     6.0
8146     6.0
        ... 
34563    6.0
34564    6.0
34565    6.0
34566    6.0
34567    6.0
Name: cylinders, Length: 206, dtype: float64

In [102]:
#4. Clip the values of a numeric series to between the 10th and 90th percent
cyl.dropna().clip(cyl.quantile(.1), cyl.quantile(.9)).sort_values()

0        4.0
23280    4.0
23279    4.0
23278    4.0
23277    4.0
        ... 
4133     8.0
15683    8.0
22317    8.0
8284     8.0
21338    8.0
Name: cylinders, Length: 40938, dtype: float64

In [143]:
#5. Using a categorical column, replace any values that is not
# in the top 5 /10 most frequent with 'Other'
cats = pd.CategoricalDtype(set(cyl.fillna('n/a')))
cyl_cats = cyl.fillna('n/a').astype(cats)
topX = cyl_cats.value_counts()[:6]
cyl_cats_topX = pd.Series(np.select([cyl_cats.isin(topX.index)], [cyl_cats], 'Other'))
cyl_cats_topX.value_counts()


4.0      15938
6.0      14284
8.0       8801
5.0        771
12.0       626
Other      445
3.0        279
dtype: int64

In [148]:
#8. Make a function that takes a categorical series and number(n)
# and returns a replace series that replaces any value that is not
# in the top n most frequent values with 'Other'

def top_n_other(s, n):
    top_cats = s.value_counts()[:n]
    return pd.Series(np.select([s.isin(top_cats.index)], [s], 'Other'))

top_n_other(cyl_cats, 6).value_counts()
    


4.0      15938
6.0      14284
8.0       8801
5.0        771
12.0       626
Other      445
3.0        279
dtype: int64

In [153]:
#9. Using a numeric column, bin it into 10 groups that have
# the same width
pd.cut(highway_mpg, 10).value_counts()

(20.5, 32.0]      25904
(8.885, 20.5]     11583
(32.0, 43.5]       3231
(43.5, 55.0]        226
(89.5, 101.0]        83
(101.0, 112.5]       62
(55.0, 66.5]         20
(66.5, 78.0]         14
(78.0, 89.5]         11
(112.5, 124.0]       10
Name: highway08, dtype: int64

In [155]:
#10 Using a numeric column, bin it into 10 groups that have
# equal sized bins
pd.qcut(highway_mpg, 10).value_counts()

(22.0, 24.0]     6197
(8.999, 17.0]    5548
(25.0, 27.0]     5369
(19.0, 21.0]     4326
(27.0, 29.0]     3815
(32.0, 124.0]    3657
(17.0, 19.0]     3440
(29.0, 32.0]     3254
(21.0, 22.0]     3156
(24.0, 25.0]     2382
Name: highway08, dtype: int64